## clone repo & download dataset

In [ ]:
!git clone https://github.com/Yui-Arthur/TFLM_with_keyword_spotting.git
%cd TFLM_with_keyword_spotting

In [ ]:
!mkdir speech_commands
!wget http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz
!tar -C speech_commands -zxvf speech_commands_v0.02.tar.gz > /dev/null

## training

In [ ]:
from pathlib import Path
from tqdm import tqdm
import numpy as np
import logging
import sys   
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from train_and_convert_utils import logger_setting , google_speech_commands_dataset , show_train_results , testing_model , train_one_epoch , conv_model


In [ ]:
input_dim = (16000,1)
wav_size = 16000
output_class = 35
# hyperparameter
epochs = 10
batch_size = 64
learning_rate = 1e-3

# path & logger setting
speech_commands_root_folder = Path("./speech_commands")
root_folder = Path("train")
logger : logging.Logger = logger_setting(root_folder , "colab")

In [ ]:
# model init
model = conv_model(input_dim , output_class , learning_rate)
model.build((batch_size,16000,1))
model._model.summary()
# exit()
# logger.info(f"Model Parameters : {num_params}")

# get dataloader
# train_dataloader , valid_dataloader , test_dataloader = google_speech_commands_dataset(speech_commands_root_folder, wav_size, batch_size ,logger , load_in_memory=True)
train_dataloader , valid_dataloader , test_dataloader = google_speech_commands_dataset(speech_commands_root_folder, wav_size, batch_size , logger, (64,64,64))

# train / valid
train_info = []
max_acc = -1
for epoch in range(epochs):
    logger.info(f"epoch {epoch} :")
    epoch_info = train_one_epoch(model , train_dataloader , valid_dataloader , max_acc , logger , root_folder)
    train_info.append(list(epoch_info))
    max_acc = max(epoch_info[2] ,max_acc)

# show train the result    
show_train_results(train_info , root_folder)

del train_dataloader , valid_dataloader , test_dataloader , model

## convert and testing

In [ ]:
from train_and_convert_utils import save_model_tflite_quant , test_tflite_model

# test keras model
_, _, test_dataset = google_speech_commands_dataset(speech_commands_root_folder , wav_size, batch_size, logger, (1,1,int(1e9)), True)
testing_model(test_dataset , logger , root_dir=root_folder)
del _, test_dataset

# # test the tflite model
_, _, test_dataset = google_speech_commands_dataset(speech_commands_root_folder , wav_size, 1, logger, (1,1,int(1e9)), True)
save_model_tflite_quant(root_folder / "model", test_dataset, logger)
test_tflite_model(root_folder / "model" / "float_model.tflite", test_dataset , logger)
test_tflite_model(root_folder / "model" / "quant_model.tflite", test_dataset , logger)
del _, test_dataset